<a href="https://colab.research.google.com/github/SodisettiRakesh123/cognifyz/blob/main/Task3_cogi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!unzip /content/archive\ \(19\).zip

Archive:  /content/archive (19).zip
  inflating: swiggy.csv              


In [2]:
import pandas as pd

df = pd.read_csv('swiggy.csv')
print("✅ Dataset loaded successfully!")
print(df.head())
print(df.info())

✅ Dataset loaded successfully!
    ID         Area       City         Restaurant  Price  Avg ratings  \
0  211  Koramangala  Bangalore        Tandoor Hut  300.0          4.4   
1  221  Koramangala  Bangalore      Tunday Kababi  300.0          4.1   
2  246    Jogupalya  Bangalore            Kim Lee  650.0          4.4   
3  248  Indiranagar  Bangalore  New Punjabi Hotel  250.0          3.9   
4  249  Indiranagar  Bangalore                Nh8  350.0          4.0   

   Total ratings                                          Food type  \
0            100          Biryani,Chinese,North Indian,South Indian   
1            100                                   Mughlai,Lucknowi   
2            100                                            Chinese   
3            500               North Indian,Punjabi,Tandoor,Chinese   
4             50  Rajasthani,Gujarati,North Indian,Snacks,Desser...   

        Address  Delivery time  
0     5Th Block             59  
1     5Th Block             56  
2   

# Task
Analyze the provided dataset "/content/archive (19).zip" to perform cuisine classification. Identify the target variable and features, preprocess the data, train a classification model, and evaluate its performance.

In [10]:
# Define the target variable
target = df['Food type']

# Select features, excluding 'ID' and the target variable
features = df.drop(['ID', 'Food type'], axis=1)

print("✅ Target variable and features defined successfully!")
print("\nTarget variable shape:", target.shape)
print("\nFeatures shape:", features.shape)
print("\nFirst 5 rows of features:")
print(features.head())

✅ Target variable and features defined successfully!

Target variable shape: (8680,)

Features shape: (8680, 8)

First 5 rows of features:
  Area City Restaurant  Price  Avg ratings  Total ratings Address  \
0  353    1       6719  300.0          4.4            100      44   
1  353    1       7420  300.0          4.1            100      44   
2  292    1       3606  650.0          4.4            100     485   
3  272    1       4751  250.0          3.9            500      56   
4  272    1       4782  350.0          4.0             50      56   

   Delivery time  
0             59  
1             56  
2             50  
3             57  
4             63  


In [11]:
# Examine the unique values and structure of the 'Food type' column
print("Unique values in 'Food type' column:")
print(target.unique()[:20]) # Print the first 20 unique values to get an idea of the format
print("\nTotal number of unique 'Food type' combinations:", target.nunique())

# Check a few examples with multiple food types
print("\nExamples of 'Food type' entries with multiple cuisines:")
print(target.iloc[0])
print(target.iloc[3])
print(target.iloc[4])

Unique values in 'Food type' column:
[518 2500 751 2953 3258 2501 2451 1390 61 296 1159 2653 2502 32 1493 2382
 313 3281 2514 2479]

Total number of unique 'Food type' combinations: 3734

Examples of 'Food type' entries with multiple cuisines:
518
2953
3258


In [12]:
from sklearn.preprocessing import MultiLabelBinarizer

# Retrieve the original 'Food type' column from the dataframe before encoding
original_food_type = pd.read_csv('swiggy.csv')['Food type']

# Split the 'Food type' strings by comma and convert to a list of lists
food_types_list = original_food_type.apply(lambda x: x.split(','))

# Initialize MultiLabelBinarizer
mlb = MultiLabelBinarizer()

# Fit and transform the food types
target_encoded = mlb.fit_transform(food_types_list)

# Convert the resulting numpy array to a DataFrame for better readability and handling
target_encoded_df = pd.DataFrame(target_encoded, columns=mlb.classes_)

print("\n✅ 'Food type' column processed for multi-label classification successfully!")
print("\nShape of encoded target variable:", target_encoded_df.shape)
print("\nFirst 5 rows of encoded target variable:")
print(target_encoded_df.head())
print("\nClasses (unique food types) encoded:")
print(mlb.classes_[:20]) # Print first 20 classes


✅ 'Food type' column processed for multi-label classification successfully!

Shape of encoded target variable: (8680, 601)

First 5 rows of encoded target variable:
    Keventers & more  Afghani  African  American  \
0                  0        0        0         0   
1                  0        0        0         0   
2                  0        0        0         0   
3                  0        0        0         0   
4                  0        0        0         0   

   American  Beverages  Fast Food  American  Chinese  Continental  Desserts  \
0                               0                                         0   
1                               0                                         0   
2                               0                                         0   
3                               0                                         0   
4                               0                                         0   

   American  Chinese  Indian  American  Contin

In [13]:
# Check data types of the features DataFrame
print("Data types of features before final check:")
print(features.dtypes)

# If 'Food type' is still in features, remove it (it was meant to be the target)
if 'Food type' in features.columns:
    features = features.drop('Food type', axis=1)
    print("\nRemoved 'Food type' from features.")

# Check data types again after potentially removing 'Food type'
print("\nData types of features after removing 'Food type':")
print(features.dtypes)

# Verify if all columns are numerical
# The previous Label Encoding step should have converted categorical columns to integers.
# We need to ensure no object types remain.
non_numerical_cols = features.select_dtypes(include='object').columns

if len(non_numerical_cols) > 0:
    print(f"\nFound non-numerical columns: {list(non_numerical_cols)}")
    # Re-apply Label Encoding to ensure they are numerical if any remain
    # This is a safeguard, assuming the previous step intended to encode these.
    from sklearn.preprocessing import LabelEncoder
    label_encoders_check = {}
    for col in non_numerical_cols:
         le = LabelEncoder()
         features.loc[:, col] = le.fit_transform(features[col].astype(str))
         label_encoders_check[col] = le
    print("Re-applied Label Encoding to remaining non-numerical columns.")
    print("\nData types after re-encoding:")
    print(features.dtypes)
else:
    print("\nAll features are already in numerical format.")

# Identify numerical columns (should be all columns at this point)
numerical_cols = features.select_dtypes(include=np.number).columns
print("\nNumerical columns in features:")
print(list(numerical_cols))

# Scaling is not applied for RandomForestRegressor, so this step is skipped.
print("\nScaling is not required for RandomForestRegressor, skipping scaling.")

# Confirm all columns are numerical
print("\nFinal check on data types:")
print(features.dtypes)

Data types of features before final check:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Data types of features after removing 'Food type':
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Found non-numerical columns: ['Area', 'City', 'Restaurant', 'Address']
Re-applied Label Encoding to remaining non-numerical columns.

Data types after re-encoding:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Numerical columns in features:
['Price', 'Avg ratings', 'Total ratings', 'Delivery time']

Scalin

In [14]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Identify non-numerical columns again
non_numerical_cols = features.select_dtypes(include='object').columns

if len(non_numerical_cols) > 0:
    print(f"Attempting to re-encode non-numerical columns: {list(non_numerical_cols)}")
    label_encoders_check = {}
    for col in non_numerical_cols:
         le = LabelEncoder()
         features.loc[:, col] = le.fit_transform(features[col].astype(str))
         label_encoders_check[col] = le
         # Explicitly check and potentially convert dtype after encoding
         if features[col].dtype == 'object':
             try:
                 features.loc[:, col] = pd.to_numeric(features[col])
                 print(f"Converted column '{col}' to numerical after encoding.")
             except ValueError:
                 print(f"Could not convert column '{col}' to numerical after encoding.")

    print("\nData types after attempting re-encoding and conversion:")
    print(features.dtypes)
else:
    print("\nNo non-numerical columns found requiring encoding.")

# Identify numerical columns (should be all columns at this point)
numerical_cols = features.select_dtypes(include=np.number).columns
print("\nNumerical columns in features:")
print(list(numerical_cols))

# Confirm all columns are numerical
print("\nFinal check on data types:")
print(features.dtypes)

Attempting to re-encode non-numerical columns: ['Area', 'City', 'Restaurant', 'Address']
Converted column 'Area' to numerical after encoding.
Converted column 'City' to numerical after encoding.
Converted column 'Restaurant' to numerical after encoding.
Converted column 'Address' to numerical after encoding.

Data types after attempting re-encoding and conversion:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Numerical columns in features:
['Price', 'Avg ratings', 'Total ratings', 'Delivery time']

Final check on data types:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object


In [15]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Use the global df, which should have been updated with label encoding in a previous step (cell ea4e8185)
# Define features by dropping the target variable 'Food type' and the 'ID' column
features = df.drop(['ID', 'Food type'], axis=1)

print("Features DataFrame based on global df:")
print(features.head())
print("\nInitial data types of features:")
print(features.dtypes)

# Identify non-numerical columns
non_numerical_cols = features.select_dtypes(include='object').columns

if len(non_numerical_cols) > 0:
    print(f"\nFound non-numerical columns after re-initializing features: {list(non_numerical_cols)}")
    # Re-apply Label Encoding to ensure they are numerical if any remain
    # This step should use the label_encoders dictionary created in cell ea4e8185
    # However, to be safe and self-contained within the subtask logic, I will re-initialize and fit.
    label_encoders_recheck = {}
    for col in non_numerical_cols:
         le = LabelEncoder()
         features.loc[:, col] = le.fit_transform(features[col].astype(str))
         label_encoders_recheck[col] = le
    print("Re-applied Label Encoding to non-numerical columns within the subtask context.")

print("\nData types after re-applying Label Encoding:")
print(features.dtypes)

# Identify numerical columns (should be all columns at this point)
numerical_cols = features.select_dtypes(include=np.number).columns
print("\nNumerical columns in features:")
print(list(numerical_cols))

# Confirm all columns are numerical
if len(features.select_dtypes(include='object').columns) == 0:
    print("\n✅ All features are now in numerical format.")
else:
    print("\n❌ Some features are still not in numerical format.")
    print("Remaining non-numerical columns:", list(features.select_dtypes(include='object').columns))

# Scaling is not applied for RandomForestRegressor, so this step is skipped.
print("\nScaling is not required for RandomForestRegressor, skipping scaling.")

Features DataFrame based on global df:
  Area City Restaurant  Price  Avg ratings  Total ratings Address  \
0  353    1       6719  300.0          4.4            100      44   
1  353    1       7420  300.0          4.1            100      44   
2  292    1       3606  650.0          4.4            100     485   
3  272    1       4751  250.0          3.9            500      56   
4  272    1       4782  350.0          4.0             50      56   

   Delivery time  
0             59  
1             56  
2             50  
3             57  
4             63  

Initial data types of features:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Found non-numerical columns after re-initializing features: ['Area', 'City', 'Restaurant', 'Address']
Re-applied Label Encoding to non-numerical columns within the subtask context.

Da

In [16]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Re-initialize features from the global df
features = df.drop(['ID', 'Food type'], axis=1)

print("Features DataFrame based on global df:")
print(features.head())
print("\nInitial data types of features:")
print(features.dtypes)

# Identify non-numerical columns
non_numerical_cols = features.select_dtypes(include='object').columns

if len(non_numerical_cols) > 0:
    print(f"\nFound non-numerical columns: {list(non_numerical_cols)}")
    label_encoders_final = {}
    for col in non_numerical_cols:
        le = LabelEncoder()
        # Apply Label Encoding
        features.loc[:, col] = le.fit_transform(features[col].astype(str))
        label_encoders_final[col] = le
        # Explicitly convert to int64 after encoding
        features.loc[:, col] = features[col].astype('int64')
        print(f"Encoded and converted column '{col}' to int64.")

print("\nData types after encoding and explicit conversion:")
print(features.dtypes)

# Identify numerical columns (should be all columns at this point)
numerical_cols = features.select_dtypes(include=np.number).columns
print("\nNumerical columns in features:")
print(list(numerical_cols))

# Confirm all columns are numerical
if len(features.select_dtypes(include='object').columns) == 0:
    print("\n✅ All features are now in numerical format.")
else:
    print("\n❌ Some features are still not in numerical format.")
    print("Remaining non-numerical columns:", list(features.select_dtypes(include='object').columns))

# Scaling is not applied for RandomForestRegressor, so this step is skipped.
print("\nScaling is not required for RandomForestRegressor, skipping scaling.")

Features DataFrame based on global df:
  Area City Restaurant  Price  Avg ratings  Total ratings Address  \
0  353    1       6719  300.0          4.4            100      44   
1  353    1       7420  300.0          4.1            100      44   
2  292    1       3606  650.0          4.4            100     485   
3  272    1       4751  250.0          3.9            500      56   
4  272    1       4782  350.0          4.0             50      56   

   Delivery time  
0             59  
1             56  
2             50  
3             57  
4             63  

Initial data types of features:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Found non-numerical columns: ['Area', 'City', 'Restaurant', 'Address']
Encoded and converted column 'Area' to int64.
Encoded and converted column 'City' to int64.
Encoded and converted

In [17]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Re-initialize features from the global df to ensure we start fresh within this cell
features = df.drop(['ID', 'Food type'], axis=1)

print("Features DataFrame based on global df:")
display(features.head()) # Use display for better output in notebook
print("\nInitial data types of features:")
print(features.dtypes)

# Identify non-numerical columns
non_numerical_cols = features.select_dtypes(include='object').columns

if len(non_numerical_cols) > 0:
    print(f"\nFound non-numerical columns: {list(non_numerical_cols)}")
    label_encoders_final = {}
    for col in non_numerical_cols:
        le = LabelEncoder()
        # Apply Label Encoding and update the column in place
        features[col] = le.fit_transform(features[col].astype(str))
        label_encoders_final[col] = le
        # Explicitly convert to int64 using .loc for clarity and to ensure update
        features.loc[:, col] = features[col].astype('int64')
        print(f"Encoded and converted column '{col}' to int64.")

print("\nData types after encoding and explicit conversion:")
print(features.dtypes)

# Identify numerical columns using a broader check for numeric types
numerical_cols = features.select_dtypes(include=np.number).columns
print("\nNumerical columns in features:")
print(list(numerical_cols))

# Confirm all columns are numerical by checking if any non-numerical columns remain
if len(features.select_dtypes(exclude=np.number).columns) == 0:
    print("\n✅ All features are now in numerical format.")
else:
    print("\n❌ Some features are still not in numerical format.")
    print("Remaining non-numerical columns:", list(features.select_dtypes(exclude=np.number).columns))


# Scaling is not applied for RandomForestRegressor, so this step is skipped.
print("\nScaling is not required for RandomForestRegressor, skipping scaling.")

Features DataFrame based on global df:


,Area,City,Restaurant,Price,Avg ratings,Total ratings,Address,Delivery time
0,353,1,6719,300.0,4.4,100,44,59
1,353,1,7420,300.0,4.1,100,44,56
2,292,1,3606,650.0,4.4,100,485,50
3,272,1,4751,250.0,3.9,500,56,57
4,272,1,4782,350.0,4.0,50,56,63



Initial data types of features:
Area              object
City              object
Restaurant        object
Price            float64
Avg ratings      float64
Total ratings      int64
Address           object
Delivery time      int64
dtype: object

Found non-numerical columns: ['Area', 'City', 'Restaurant', 'Address']
Encoded and converted column 'Area' to int64.
Encoded and converted column 'City' to int64.
Encoded and converted column 'Restaurant' to int64.
Encoded and converted column 'Address' to int64.

Data types after encoding and explicit conversion:
Area               int64
City               int64
Restaurant         int64
Price            float64
Avg ratings      float64
Total ratings      int64
Address            int64
Delivery time      int64
dtype: object

Numerical columns in features:
['Area', 'City', 'Restaurant', 'Price', 'Avg ratings', 'Total ratings', 'Address', 'Delivery time']

✅ All features are now in numerical format.

Scaling is not required for RandomForestRegr

In [18]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target_encoded_df, test_size=0.2, random_state=42)

print("✅ Data split successfully!")
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

✅ Data split successfully!
X_train shape: (6944, 8)
X_test shape: (1736, 8)
y_train shape: (6944, 601)
y_test shape: (1736, 601)


In [19]:
from sklearn.ensemble import RandomForestClassifier

# Instantiate the RandomForestClassifier model
# Using a random_state for reproducibility
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model on the training data
# X_train are the features, y_train is the multi-label target
model.fit(X_train, y_train)

print("\n✅ RandomForestClassifier model trained successfully!")


✅ RandomForestClassifier model trained successfully!


In [20]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model using multi-label classification metrics
# 'micro': Calculate metrics globally by counting the total true positives, false negatives, and false positives.
# 'macro': Calculate metrics for each label, and find their unweighted mean.
# 'weighted': Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label).

accuracy = accuracy_score(y_test, y_pred)
precision_micro = precision_score(y_test, y_pred, average='micro')
recall_micro = recall_score(y_test, y_pred, average='micro')
f1_micro = f1_score(y_test, y_pred, average='micro')

precision_macro = precision_score(y_test, y_pred, average='macro')
recall_macro = recall_score(y_test, y_pred, average='macro')
f1_macro = f1_score(y_test, y_pred, average='macro')

precision_weighted = precision_score(y_test, y_pred, average='weighted')
recall_weighted = recall_score(y_test, y_pred, average='weighted')
f1_weighted = f1_score(y_test, y_pred, average='weighted')


print(f"Accuracy: {accuracy:.4f}")
print("\nMicro-averaged metrics:")
print(f"Precision (Micro): {precision_micro:.4f}")
print(f"Recall (Micro): {recall_micro:.4f}")
print(f"F1-score (Micro): {f1_micro:.4f}")

print("\nMacro-averaged metrics:")
print(f"Precision (Macro): {precision_macro:.4f}")
print(f"Recall (Macro): {recall_macro:.4f}")
print(f"F1-score (Macro): {f1_macro:.4f}")

print("\nWeighted-averaged metrics:")
print(f"Precision (Weighted): {precision_weighted:.4f}")
print(f"Recall (Weighted): {recall_weighted:.4f}")
print(f"F1-score (Weighted): {f1_weighted:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/me

Accuracy: 0.0167

Micro-averaged metrics:
Precision (Micro): 0.4898
Recall (Micro): 0.0462
F1-score (Micro): 0.0845

Macro-averaged metrics:
Precision (Macro): 0.0217
Recall (Macro): 0.0031
F1-score (Macro): 0.0041

Weighted-averaged metrics:
Precision (Weighted): 0.4010
Recall (Weighted): 0.0462
F1-score (Weighted): 0.0767


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [21]:
from sklearn.metrics import multilabel_confusion_matrix

# Calculate the confusion matrix for each label
# This will return a list of confusion matrices, one for each label
confusion_matrices = multilabel_confusion_matrix(y_test, y_pred)

print("\n📊 Confusion Matrix for each label:")
# You can iterate through the confusion matrices to see the results for each cuisine type
# Due to the large number of labels (601), printing all of them might be too verbose.
# Let's print the confusion matrix for the first few labels as an example.
# We'll also map the index back to the food type using mlb.classes_ if mlb is available.

if 'mlb' in globals(): # Check if mlb (MultiLabelBinarizer) is available
    for i, matrix in enumerate(confusion_matrices[:10]): # Print for the first 10 labels
        label = mlb.classes_[i]
        print(f"\nConfusion Matrix for '{label}':")
        print(matrix)
else:
    print("\nMultiLabelBinarizer object 'mlb' not found. Printing confusion matrices by index.")
    for i, matrix in enumerate(confusion_matrices[:10]): # Print for the first 10 labels by index
        print(f"\nConfusion Matrix for label index {i}:")
        print(matrix)



📊 Confusion Matrix for each label:

Confusion Matrix for ' Keventers & more':
[[1736    0]
 [   0    0]]

Confusion Matrix for 'Afghani':
[[1734    0]
 [   2    0]]

Confusion Matrix for 'African':
[[1735    0]
 [   1    0]]

Confusion Matrix for 'American':
[[1675    0]
 [  61    0]]

Confusion Matrix for 'American  Beverages  Fast Food':
[[1736    0]
 [   0    0]]

Confusion Matrix for 'American  Chinese  Continental  Desserts':
[[1736    0]
 [   0    0]]

Confusion Matrix for 'American  Chinese  Indian':
[[1736    0]
 [   0    0]]

Confusion Matrix for 'American  Continental':
[[1735    0]
 [   1    0]]

Confusion Matrix for 'American  Continental  Italian  Pastas  Snacks  Desserts  Beverages':
[[1735    0]
 [   1    0]]

Confusion Matrix for 'American  European  Continental':
[[1736    0]
 [   0    0]]


In [22]:
# Check if the trained model object has the feature_importances_ attribute
if hasattr(model, 'feature_importances_'):
    print("\n✅ Model has feature_importances_ attribute.")

    # Retrieve the feature importances
    importances = model.feature_importances_

    # Create a pandas DataFrame to display the feature names and their corresponding importances
    # Ensure the features DataFrame is correctly aligned with the model's training features
    feature_names = X_train.columns
    feature_importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importances
    })

    # Sort the DataFrame by importance in descending order
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

    # Print or display the sorted feature importance DataFrame
    print("\n🔥 Feature Importance from RandomForestClassifier:")
    display(feature_importance_df)
else:
    print("\n❌ Model does not have feature_importances_ attribute.")


✅ Model has feature_importances_ attribute.

🔥 Feature Importance from RandomForestClassifier:


,Feature,Importance
2,Restaurant,0.208533
7,Delivery time,0.171179
6,Address,0.151373
0,Area,0.145028
3,Price,0.116156
4,Avg ratings,0.087400
1,City,0.069601
5,Total ratings,0.050729
